<a href="https://colab.research.google.com/github/harry-woo/S-OIL/blob/main/%EC%9A%B0%ED%98%95%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어처리 기말고사


**본 코드는 네이버의 영화 리뷰 데이터를 기반으로 이 리뷰가 긍정인지 부정인지를 예측하는 모델을 작성하는 것입니다.<br> 본 코드는 기본적인 모델로만 학습하여 예측을 하는 코드이며, CNN 및 RNN 모델을 생성하여 모델의 정확도를 높여야 합니다.**

아래와 같은 사항을 수정하여 기말고사의 점수를 획득할 수 있습니다.


1. 데이터 전처리 함수 (+10)
1. 데이터 토큰화 방법 (+10)
1. 모델 수정(+20)
1. 모델에 관한 설명(+20)
1. 테스트 정확도가 제시한 코드보다 높을 경우(+20) 테스트 정확도 ---- 0.8023
1. 테스트 정확도의 상대평가 (+20)


유의 사항 <br>

* 반드시 ipynb 확장자로 제출할 것
* 타인과 수정된 코드 공유 금지


In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 450kB 40.9MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import numpy as np
import pandas as pd
import os
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


**학습데이터셋을 ratings_train 하나만 가지고 옴**

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x7f0a66ebb410>)

**학습데이터 확인해 보기**

In [4]:
train_data = pd.read_csv('ratings_train.txt', header=0, delimiter='\t', quoting=3)
train_data.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


**전처리 함수**

(전처리 함수 수정시 +10)

In [5]:
from konlpy.tag import Okt
import re

# 감탄사, 조사들은 제거할 수 있도록 정의
# RANKS NL의 불용어 리스트(https://www.ranks.nl/stopwords/korean)를 기준으로 전처리 함수 수정
stop_words = set(['아','휴','아이구','아이쿠','아이고','어','나','우리','저희','따라','의해','을','를','에','의','가','으로','로','에게','뿐이다','의거하여','근거하여','입각하여','기준으로','예하면','예를 들면','예를 들자면','저','소인','소생','저희','지말고','하지마','하지마라','다른','물론','또한','그리고','비길수 없다','해서는 안된다','뿐만 아니라','만이 아니다','만은 아니다','막론하고','관계없이','그치지 않다','그러나','그런데','하지만','든간에','논하지 않다','따지지 않다','설사','비록','더라도','아니면','만 못하다','하는 편이 낫다','불문하고','향하여','향해서','향하다','쪽으로','틈타','이용하여','타다','오르다','제외하고','이 외에','이 밖에','하여야','비로소','한다면 몰라도','외에도','이곳','여기','부터','기점으로','따라서','할 생각이다','하려고하다','이리하여','그리하여','그렇게 함으로써','하지만','일때','할때','앞에서','중에서','보는데서','으로써','로써','까지','해야한다','일것이다','반드시','할줄알다','할수있다','할수있어','임에 틀림없다','한다면','등','등등','제','겨우','단지','다만','할뿐','딩동','댕그','대해서','대하여','대하면','훨씬','얼마나','얼마만큼','얼마큼','남짓','여','얼마간','약간','다소','좀','조금','다수','몇','얼마','지만','하물며','또한','그러나','그렇지만','하지만','이외에도','대해 말하자면','뿐이다','다음에','반대로','반대로 말하자면','이와 반대로','바꾸어서 말하면','바꾸어서 한다면','만약','그렇지않으면','까악','툭','딱','삐걱거리다','보드득','비걱거리다','꽈당','응당','해야한다','에 가서','각','각각','여러분','각종','각자','제각기','하도록하다','와','과','그러므로','그래서','고로','한 까닭에','하기 때문에','거니와','이지만','대하여','관하여','관한','과연','실로','아니나다를가','생각한대로','진짜로','한적이있다','하곤하였다','하','하하','허허','아하','거바','와','오','왜','어째서','무엇때문에','어찌','하겠는가','무슨','어디','어느곳','더군다나','하물며','더욱이는','어느때','언제','야','이봐','어이','여보시오','흐흐','흥','휴','헉헉','헐떡헐떡','영차','여차','어기여차','끙끙','아야','앗','아야','콸콸','졸졸','좍좍','뚝뚝','주룩주룩','솨','우르르','그래도','또','그리고','바꾸어말하면','바꾸어말하자면','혹은','혹시','답다','및','그에 따르는','때가 되어','즉','지든지','설령','가령','하더라도','할지라도','일지라도','지든지','몇','거의','하마터면','인젠','이젠','된바에야','된이상','만큼','어찌됏든','그위에','게다가','점에서 보아','비추어 보아','고려하면','하게될것이다','일것이다','비교적','좀','보다더','비하면','시키다','하게하다','할만하다','의해서','연이서','이어서','잇따라','뒤따라','뒤이어','결국','의지하여','기대여','통하여','자마자','더욱더','불구하고','얼마든지','마음대로','주저하지 않고','곧','즉시','바로','당장','하자마자','밖에 안된다','하면된다','그래','그렇지','요컨대','다시 말하자면','바꿔 말하면','즉','구체적으로','말하자면','시작하여','시초에','이상','허','헉','허걱','바와같이','해도좋다','해도된다','게다가','더구나','하물며','와르르','팍','퍽','펄렁','동안','이래','하고있었다','이었다','에서','로부터','까지','예하면','했어요','해요','함께','같이','더불어','마저','마저도','양자','모두','습니다','가까스로','하려고하다','즈음하여','다른','다른 방면으로','해봐요','습니까','했어요','말할것도 없고','무릎쓰고','개의치않고','하는것만 못하다','하는것이 낫다','매','매번','들','모','어느것','어느','로써','갖고말하자면','어디','어느쪽','어느것','어느해','어느 년도','라 해도','언젠가','어떤것','어느것','저기','저쪽','저것','그때','그럼','그러면','요만한걸','그래','그때','저것만큼','그저','이르기까지','할 줄 안다','할 힘이 있다','너','너희','당신','어찌','설마','차라리','할지언정','할지라도','할망정','할지언정','구토하다','게우다','토하다','메쓰겁다','옆사람','퉤','쳇','의거하여','근거하여','의해','따라','힘입어','그','다음','버금','두번째로','기타','첫번째로','나머지는','그중에서','견지에서','형식으로 쓰여','입장에서','위해서','단지','의해되다','하도록시키다','뿐만아니라','반대로','전후','전자','앞의것','잠시','잠깐','하면서','그렇지만','다음에','그러한즉','그런즉','남들','아무거나','어찌하든지','같다','비슷하다','예컨대','이럴정도로','어떻게','만약','만일','위에서 서술한바와같이','인 듯하다','하지 않는다면','만약에','무엇','무슨','어느','어떤','아래윗','조차','한데','그럼에도 불구하고','여전히','심지어','까지도','조차도','하지 않도록','않기 위하여','때','시각','무렵','시간','동안','어때','어떠한','하여금','네','예','우선','누구','누가 알겠는가','아무도','줄은모른다','줄은 몰랏다','하는 김에','겸사겸사','하는바','그런 까닭에','한 이유는','그러니','그러니까','때문에','그','너희','그들','너희들','타인','것','것들','너','위하여','공동으로','동시에','하기 위하여','어찌하여','무엇때문에','붕붕','윙윙','나','우리','엉엉','휘익','윙윙','오호','아하','어쨋든','만 못하다','하기보다는','차라리','하는 편이 낫다','흐흐','놀라다','상대적으로 말하자면','마치','아니라면','쉿','그렇지 않으면','그렇지 않다면','안 그러면','아니었다면','하든지','아니면','이라면','좋아','알았어','하는것도','그만이다','어쩔수 없다','하나','일','일반적으로','일단','한켠으로는','오자마자','이렇게되면','이와같다면','전부','한마디','한항목','근거로','하기에','아울러','하지 않도록','않기 위해서','이르기까지','이 되다','로 인하여','까닭으로','이유만으로','이로 인하여','그래서','이 때문에','그러므로','그런 까닭에','알 수 있다','결론을 낼 수 있다','으로 인하여','있다','어떤것','관계가 있다','관련이 있다','연관되다','어떤것들','에 대해','이리하여','그리하여','여부','하기보다는','하느니','하면 할수록','운운','이러이러하다','하구나','하도다','다시말하면','다음으로','에 있다','에 달려 있다','우리','우리들','오히려','하기는한데','어떻게','어떻해','어찌됏어','어때','어째서','본대로','자','이','이쪽','여기','이것','이번','이렇게말하자면','이런','이러한','이와 같은','요만큼','요만한 것','얼마 안 되는 것','이만큼','이 정도의','이렇게 많은 것','이와 같다','이때','이렇구나','것과 같이','끼익','삐걱','따위','와 같은 사람들','부류의 사람들','왜냐하면','중의하나','오직','오로지','에 한하다','하기만 하면','도착하다','까지 미치다','도달하다','정도에 이르다','할 지경이다','결과에 이르다','관해서는','여러분','하고 있다','한 후','혼자','자기','자기집','자신','우에 종합한것과같이','총적으로 보면','총적으로 말하면','총적으로','대로 하다','으로서','참','그만이다','할 따름이다','쿵','탕탕','쾅쾅','둥둥','봐','봐라','아이야','아니','와아','응','아이','참나','년','월','일','령','영','일','이','삼','사','오','육','륙','칠','팔','구','이천육','이천칠','이천팔','이천구','하나','둘','셋','넷','다섯','여섯','일곱','여덟','아홉','령','영'])

okt=Okt()

def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    #  문장에서 한글만 뽑아냄
    review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", review)
    
    # okt 라이브러리로 단어의 어간별로 분리
    word_review = okt.morphs(review_text, stem=True)
    
    #  감탄사와 조사 제거
    if remove_stopwords:
        word_review = [token for token in word_review if token not in stop_words]
        
    return word_review

**학습 데이터 전처리 수행**


In [6]:
clean_train_review = []

for review in train_data['document']:
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, True, stop_words))
    else:
        clean_train_review.append([])

**토큰화** <br>
토큰 방법을 수정시 + 10

In [7]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

# 전체 순서를 만들고
tokenizer.fit_on_texts(clean_train_review)

#  단어의 위치를 숫자로 표시
train_sequence = tokenizer.texts_to_sequences(clean_train_review)

# 패딩 기준 조정
MAX_SEQUENCE_LENGTH = 30

# 최대 문장 길이는 30으로 padding
train_inputs = pad_sequences(train_sequence, maxlen = MAX_SEQUENCE_LENGTH, padding='post')

#  train, test label 만들기
train_labels = np.array(train_data['label'])

**모델 만들기**
<br> (모델 수정시 + 20)

<모델에 관한 설명을 자세하게 서술시 +20><br>


In [8]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.55.50.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.55.50.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [61]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447

#  데이터를 train과 eval로 분리
from sklearn.model_selection import train_test_split
input_train, input_eval, label_train, label_eval = train_test_split(train_inputs, train_labels, test_size=TEST_SPLIT, random_state=RNG_SEED)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, Conv1D, LSTM, GRU, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
with strategy.scope():
    model = Sequential()
    #  Word Embedding
    word_size = len(tokenizer.word_index)+1
    model.add(Embedding(word_size, 100, input_length=MAX_SEQUENCE_LENGTH))

    # DNN, CNN, RNN 방식의 구성을 테스트하였으며, 가장 성능이 높은 것으로 나타난 RNN을 기준으로 모델을 구성하였다.
    # LSTM, 양방향 LSTM, GRU 중 성능이 상대적으로 낫고 효율적인 GRU를 사용하였다.
    # 단층 모델 대비 다층 모델에서 성능이 개선되었으며, 3개층 이상에서 성능이 오히려 저하되어 2층으로만 구성하였다.
    model.add(GRU(units = 128, return_sequences=True)) # 다층구조를 구성하기 위한 파라미터 추가
    model.add(GRU(units = 128)) # GRU를 2층으로 구성하였다.
    model.add(Dense(128, activation='relu')) # Dense 레이어를 추가하며 활성함수로 relu를 사용하였다.
    model.add(Dropout(0.5)) # 과적합의 방지를 위하여 Dropout을 추가하였다.
    model.add(Dense(1, activation='sigmoid')) # 결과값 출력을 위한 Dense 레이어
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['acc'])

In [62]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [63]:
history = model.fit(input_train, label_train, epochs=20, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/20
1800/1800 [==============================] - 40s 18ms/step - loss: 0.4524 - acc: 0.7706 - val_loss: 0.3653 - val_acc: 0.8367

Epoch 00001: val_acc improved from -inf to 0.83670, saving model to best_model.h5
Epoch 2/20
1800/1800 [==============================] - 32s 18ms/step - loss: 0.3421 - acc: 0.8511 - val_loss: 0.3509 - val_acc: 0.8432

Epoch 00002: val_acc improved from 0.83670 to 0.84319, saving model to best_model.h5
Epoch 3/20
1800/1800 [==============================] - 31s 17ms/step - loss: 0.3147 - acc: 0.8674 - val_loss: 0.3378 - val_acc: 0.8514

Epoch 00003: val_acc improved from 0.84319 to 0.85141, saving model to best_model.h5
Epoch 4/20
1800/1800 [==============================] - 31s 17ms/step - loss: 0.2904 - acc: 0.8805 - val_loss: 0.3323 - val_acc: 0.8517

Epoch 00004: val_acc improved from 0.85141 to 0.85167, saving model to best_model.h5
Epoch 5/20
1800/1800 [==============================] - 31s 17ms/step - loss: 0.2679 - acc: 0.8930 - val_loss: 0.33

테스트 정확도가 아래 보다 높을 경우 +20 <br>
테스트 정확도가 과제를 수행한 사람중에 상대평가 하여 +20 추가
<br> 예) 테스트 정확도가 가장 높을 경우 20점 만점
<br>                    두번째로 높을 경우 19점
<br>                    세번째로 높을 경우 18점

In [64]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(input_eval, label_eval)[1]))

469/469 [==============================] - 7s 14ms/step - loss: 0.3331 - acc: 0.8536

 테스트 정확도: 0.8536


In [65]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stop_words] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = MAX_SEQUENCE_LENGTH) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


In [66]:
sentiment_predict("이거 졸렸음")

52.60% 확률로 부정 리뷰입니다.

